In [1]:
import nltk
import pandas as pd
import random
import numpy as np
import string
import math

from nltk import ngrams
from nltk.corpus import stopwords
from nltk.stem.snowball import FrenchStemmer
from nltk.tokenize import word_tokenize

from numpy import array
from collections import Counter
from scipy.sparse import csr_matrix

from french_lefff_lemmatizer.french_lefff_lemmatizer import FrenchLefffLemmatizer

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.cluster import KMeans

from gensim.test.utils import common_dictionary, common_corpus
from gensim.models import LsiModel
from gensim import corpora, models, utils
from gensim.test.utils import common_corpus, common_dictionary, get_tmpfile
from gensim.models import LsiModel
from gensim.corpora import Dictionary

import re

In [2]:
# Use spacy lib
# On https://spacy.io/

import spacy
nlp = spacy.load('fr')

In [3]:
##############
# Parameters #
##############

min_gram = 1
max_gram = 3

# To create ours partitions, we must first know the years which will be the limits
limit_years = [2007, 2010, 2014]

# Ignore words that appear at a frequency less than max_frequ in the corpus
max_frequ = 0.8

# Ignore words appearing less than min_appear in the whole corpus
min_appear = 5

# Number of clusters by partitions
nb_clusters = 5

In [4]:
# Datas preprocessing methods.

# Lemmatisation without poncutations

stemmer = nltk.stem.snowball.FrenchStemmer()
fstw = stopwords.words('french')

# French Stop Words, extraits depuis le fichier stopwords-fr.txt + stopwords french de nltk
sourceFST = [x.replace('\n', '') for x in open('stopwords-fr.txt', mode="r", encoding="utf-8").readlines()]+fstw
sourceFST += [x.replace('\n', '') for x in open('perso_words-fr.txt', mode="r", encoding="utf-8").readlines()]

# Based on ration of french and english stopwords
def isEnglish(article):
    total_fsw = len([x for x in article.split() if x in sourceFST])
    total_esw = len([x for x in article.split() if x in stopwords.words('english')])
    ratio = 100
    if total_fsw != 0:
        ratio = total_esw/total_fsw
    return ratio > 1 and total_esw > 3

def lemmatize(article):
    arti_lower = article.lower()
    arti_2words = re.sub(" [0-z][0-z] ", " ", arti_lower) # word of length < 2
    arti_e = re.sub("(é|è|ê)", "e", arti_2words)
    arti_o = re.sub("à", "a", arti_e)
    arti_i = re.sub("ô", "o", arti_o)
    artiregex = re.sub("î", "i", arti_i)
    output = []
    outPonc = artiregex.translate(artiregex.maketrans("","", string.punctuation))
    outLem = nlp(outPonc)
    for token in outLem:
        if token.lemma_ not in sourceFST and [x for x in token.lemma_ if x not in "0123456789"] != []:
            output.append(token.lemma_)
    res = ' '.join(output)
    return res

In [5]:
# Data Reading
data = pd.read_csv('export_articles_EGC_2004_2018.csv', sep='\t', header=0)

In [6]:
# Let's process our corpus, and determine a limit to split it in partitions

# usable[] correspond to our corpus processed
# limits[] let us know when to delimit partitions
limits = []
usable = []

prev_year = data['year'][0]
numArti = 0
for i in range(0, len(data['abstract']), 1):
    #if not null, empty, or whatever (so if there is a abstract):
    if not isinstance(data['abstract'][i], float) and not isEnglish(data['abstract'][i]):
        text = data['abstract'][i]
        if not isinstance(data['title'][i], float):
            text += " "+data['title'][i]

        numArti+=1
        usable.append(re.sub(" [0-z][0-z] ", " ", stemmer.stem(lemmatize(text))))
        year = data['year'][i]
        if year != prev_year:
            prev_year = year
            if year in limit_years:
                limits.append(numArti)
limits.append(numArti)



In [7]:
# Display pre-processed datas

vectorizer = TfidfVectorizer(stop_words=sourceFST, use_idf=True, ngram_range=(min_gram, max_gram), max_df=max_frequ, min_df=min_appear)
tfidf = vectorizer.fit_transform(usable)

print("nombre d'articles =", len(usable))
print("nombre de mots =", len(tfidf.toarray()[0]))
print("limits =", limits)

usable[1]

/info/etu/m2/i140302/venv/lib/python3.5/site-packages/sklearn/feature_extraction/text.py:286: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['quelqu'] not in stop_words.
  sorted(inconsistent))


nombre d'articles = 991
nombre de mots = 2397
limits = [223, 468, 694, 991]


'classification croise coclustering technique permettre dextraire structuresousjacente existant entrer ligne colonne tabler donneer sou former bloc application utiliser technique algorithme coclustering actuel passer lechelle approche utilise succe method modl optimiser critere vraisemblance regularisee cependent taille plaire important methode atteindre limiter article presenter nouvel algorithme coclustering niveau compter critere modl permettre traiter efficacement donnee grand tailler pouvoir memoir experience montrer lapproche propose gagn temps calcul produire solution qualite two level coclustering algorithm for very large dater set'

In [8]:
# Creation of partitions_tfidf[], which give us the TFIDF of each cluster of each partition
# partitions_tfidf[num_partition][num_doc][num_word]
# Beware, num_doc can't be equals to 1091 (max). You have partitions, so every doc aren't in every partitions
# num_word can be found via vectorizer.get_feature_name()
partitions_tfidf = []
beg = 0
for l in limits:
    last = l
    partitions_tfidf.append([list(x) for x in list(tfidf.toarray())[beg:last]])
    beg = l

In [9]:
len(partitions_tfidf)

4

# KMeans

In [10]:
# Applying KMeans on tfidf
# the labels_ give assignment of doc to the cluster number 
km = KMeans(n_clusters=nb_clusters)
km.fit(tfidf)
cluster = km.labels_

cluster_partition = [cluster[:limits[0]],cluster[limits[0]:limits[1]],cluster[limits[1]:limits[2]],cluster[limits[2]:limits[3]]]

In [11]:
part_km = []
for i in range(0, len(limits)):
    dash = km.fit(partitions_tfidf[i])
    part_km.append(dash)

In [12]:
#(part_km[1]).labels_
#
#all_labels = []
#for i in range(0,len(limits)):
#    for j in range(0, len((part_km[i]).labels_)):
        

In [13]:
# doc_clustering is a dictionnary 
# it looks like -> { doc_number : [partition_number, cluster_number] }
# This is used to reassign doc number to their respective partition and and cluster
doc_clustering = {}
for i in range(0,len(usable)):
    if i < limits[0]:
        doc_clustering[i] = [0, cluster[i]]
    elif i >= limits[0] and i < limits[1]:
        doc_clustering[i] = [1, cluster[i]]
    elif i >= limits[1] and i < limits[2]:
        doc_clustering[i] = [2, cluster[i]]
    else:
        doc_clustering[i] = [3, cluster[i]]

In [14]:
# Allows to get list of documents number
# return [dou numbers]
# params : partition_number , cluster number
partitions = []
def get_doc(part, clust):
    docs = []
    for i in range(0,len(doc_clustering)):
        if doc_clustering[i][0] == part and doc_clustering[i][1] == clust:
            docs.append(i)
    return docs

In [15]:
# Get the partitions variable
# Here partitions[part][cluster] = list of docs numbe
partitions = []
for i in range(0, len(limits)):
    clusters = []
    for j in range(0, nb_clusters):
        clusters.append(get_doc(i,j))
    partitions.append(clusters)

In [16]:
# example of output for doc_clustering
# doc 465 is in cluster 1 of the partition 1
# doc 154 is in cluster 2 of the partition 0

print(doc_clustering[465])
print(doc_clustering[154])
print()

# Here, just count docs number by cluster.
print(Counter(cluster_partition[0]))
print(Counter(cluster_partition[1]))
print(Counter(cluster_partition[2]))
print(Counter(cluster_partition[3]))

[1, 4]
[0, 0]

Counter({3: 82, 4: 77, 0: 42, 1: 17, 2: 5})
Counter({3: 78, 4: 71, 0: 66, 1: 22, 2: 8})
Counter({4: 77, 3: 76, 0: 52, 1: 12, 2: 9})
Counter({4: 105, 3: 79, 0: 62, 2: 32, 1: 19})


# Quality Measure

In [17]:
# INSERT QUALITY MEASURE HERE

# Khi²

In [18]:
# tf_of_your_word = tf[numDoc][strWord]
tf = []
for doc in usable:
    tf_doc = {}
    for word in vectorizer.get_feature_names():
        tf_doc[word] = doc.count(word)
    tf.append(tf_doc)

In [19]:
# Number total of words
# nb_total_word[numPartition]
nb_total_word = []
nb = 0

for numDoc in range(0, len(usable)):
    for word in vectorizer.get_feature_names():
        nb += tf[numDoc][word]
    if numDoc+1 in limits:
        nb_total_word.append(nb)
        nb=0
    

In [20]:
nb_total_word

[26871, 29619, 26638, 36296]

In [21]:
tf[0]

{'itemset frequent': 0,
 'confirmer': 0,
 'nouvel algorithm': 0,
 'varier': 0,
 'veiller': 0,
 'donneer luci': 0,
 'source dinformation': 0,
 'stocker': 0,
 'chaine': 0,
 'dautre': 0,
 'scenario': 0,
 'prealabl': 0,
 'comportement': 0,
 'ensembliste': 0,
 'mesurer dissimilarite': 0,
 'mecanism': 0,
 'ascendant hierarchiqu': 0,
 'evidence': 0,
 'appliquer': 0,
 'annoter': 0,
 'programmer': 0,
 'fonctionnel': 0,
 'dimager': 0,
 'ponderation': 0,
 'enregistrement': 0,
 'poser probleme': 0,
 'consommation': 0,
 'detude': 0,
 'decideur': 0,
 'mesure similarite': 0,
 'proximite': 0,
 'reduire': 0,
 'fonction croyance': 0,
 'compter': 0,
 'foret': 0,
 'volum': 0,
 'bloc': 0,
 'chemin': 0,
 'dedition': 0,
 'utiliser algorithme': 0,
 'dautre partir': 0,
 'constater': 0,
 'structuration': 0,
 'apporter': 0,
 'risquer': 0,
 'label': 0,
 'lieu': 0,
 'alternatif': 0,
 'clairement': 0,
 'grand nombre': 0,
 'analyser donnee': 0,
 'the': 0,
 'appartenir': 0,
 'modularite': 0,
 'social': 0,
 'classifie

In [22]:
# nb_word[num_partition][word]
nb_word = []

word_in_this_parti = {}
for word in vectorizer.get_feature_names():
    word_in_this_parti[word] = 0

for numDoc in range(0, len(usable)):
    for word in vectorizer.get_feature_names():
        word_in_this_parti[word] += tf[numDoc][word]
    if numDoc+1 in limits:
        nb_word.append(word_in_this_parti)
        word_in_this_parti = {}
        for word in vectorizer.get_feature_names():
            word_in_this_parti[word] = 0

In [23]:
len(nb_word)

4

In [24]:
# nb_word_by_cluster[numPartition][numCluster]
nb_word_by_cluster = []
for parti in partitions:
    nb_word_clus = []
    for cluster in parti:
        nb = 0
        for numDoc in cluster:
            for word in vectorizer.get_feature_names():
                nb += tf[numDoc][word]
        nb_word_clus.append(nb)
    nb_word_by_cluster.append(nb_word_clus)

In [25]:
# Expected values, if nothing were dependant
# exp[numPartition][numCluster][numWord]
#exp = []
#for numParti in range(0, len(partitions)):
#    exp_clus = []
#    for numCluster in range(0, len(partitions[numParti])):
#        exp_word = []
#        for numWord in range(0, vectorizer.get_feature_names()):
#            exp_word.append((nb_word[numParti][numWord] + nb_word_by_cluster[numPart][numCluster]) / nb_total_word[numParti])
#        exp_cluster.append(exp_word)
#    exp.append(exp_clus)


In [26]:
# value_of_khi2 = khi2[numPartition][numCluster][word]
khi2 = []

for numParti in range(0, len(partitions)):
    khi2parti = []
    for numCluster in range(0, len(partitions[numParti])):
        khi2cluster = {}
        
        for word in vectorizer.get_feature_names():
            word_in_this_parti[word] = 0
            E = nb_word[numParti][word]
            E =+ nb_word_by_cluster[numParti][numCluster]
            E = E/ nb_total_word[numParti]
            N = 0
            for numDoc in partitions[numParti][numCluster]:
                N += tf[numDoc][word]
            khi2cluster[word] = (pow(N - E, 2)/E)        
        khi2parti.append(khi2cluster)
    khi2.append(khi2parti)

In [27]:
# list of your labels = labels[numPartition][numCluster]
labels = []

for numPartition in range(0, len(nb_word_by_cluster)):
    label_clus = []
    for numCluster in range(0, len(nb_word_by_cluster[numPartition])):
        label_clus.append(Counter(khi2[numPartition][numCluster]).most_common(5))
    labels.append(label_clus)

In [28]:
labels

[[[('don', 79696.47369083684),
   ('donne', 74708.96544495977),
   ('donnee', 65217.38523506472),
   ('tre', 51168.33932052199),
   ('method', 14039.745617373575)],
  [('motif', 46467.02702492422),
   ('don', 23377.240827359943),
   ('donne', 20292.530407874154),
   ('quen', 16518.955306385655),
   ('donnee', 14777.612951852503)],
  [('regl', 18918.2008592758),
   ('regle', 6167.101659275798),
   ('dassoci', 4279.383259275798),
   ('associ', 4279.383259275798),
   ('voir', 4279.383259275798)],
  [('tre', 88145.476925692),
   ('don', 49887.83368244877),
   ('donne', 46224.718817583904),
   ('for', 38657.476925692026),
   ('donnee', 34814.801250016346)],
  [('tre', 90350.69861134827),
   ('don', 70843.84531034529),
   ('for', 57768.56866535466),
   ('donne', 56125.89435696713),
   ('donnee', 37591.57549881658)]],
 [[('don', 70770.2272556203),
   ('donne', 64695.664699709756),
   ('tre', 58893.63479555639),
   ('donnee', 53364.13754316022),
   ('classification', 36900.77140897492)],
  [('